In [ ]:
import sys
import subprocess
import shutil
import getpass
from pathlib import Path

from build_tools import clean

PROJECT_DIR = Path.cwd().parent
DIST_DIR = PROJECT_DIR / "dist"

print(f"Project: {PROJECT_DIR}")
print(f"Dist: {DIST_DIR}")

In [ ]:
clean.clean_tracked_notebooks(PROJECT_DIR);

## Clean Previous Build

In [ ]:
if DIST_DIR.exists():
    print(f"Removing {DIST_DIR}...")
    shutil.rmtree(DIST_DIR)
    print("[OK] Cleaned")
else:
    print("[OK] No previous build")

## Build Packages

In [ ]:
result = subprocess.run(
    [sys.executable, "-m", "build"],
    cwd=PROJECT_DIR,
    capture_output=True, text=True, encoding="utf-8", errors="replace"
)

print(result.stdout)
if result.stderr:
    print(f"STDERR: {result.stderr}")

print("\n" + "=" * 60)
if result.returncode == 0:
    print("[OK] Build succeeded!")
    for f in sorted(DIST_DIR.glob("*")):
        size_kb = f.stat().st_size / 1024
        print(f"  {f.name} ({size_kb:.1f} KB)")
else:
    print(f"[FAILED] Exit code {result.returncode}")

## Validate Packages

In [ ]:
result = subprocess.run(
    ["twine", "check"] + [str(f) for f in DIST_DIR.glob("*")],
    capture_output=True, text=True
)

print(result.stdout)
if result.returncode == 0:
    print("[OK] All packages valid")
else:
    print(f"[FAILED] Validation failed")

## Upload to PyPI

**Warning:** Once uploaded, versions cannot be deleted (only yanked).

In [ ]:
pypi_token = getpass.getpass("Enter PyPI token: ")

if not pypi_token:
    print("[CANCELLED] No token provided")
else:
    confirm = input("Type 'PUBLISH' to upload to PyPI: ")
    if confirm == "PUBLISH":
        result = subprocess.run(
            ["twine", "upload", "--username", "__token__", "--password", pypi_token]
            + [str(f) for f in DIST_DIR.glob("*")],
            capture_output=True, text=True
        )
        print(result.stdout)
        if result.stderr:
            print(result.stderr)
        if result.returncode == 0:
            print("\n[OK] Published to PyPI!")
            print("Install with: pip install snowlib")
        else:
            print(f"\n[FAILED] Upload failed")
    else:
        print("[CANCELLED]")